In [1]:
import datetime
import math
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [2]:
tweets = pd.read_csv("./data/london_tweets.csv", encoding="latin1")

In [3]:
tweets.head()

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET,datetime
0,550226818624942080,NaN,-0.262150,51.391407,-0.262150,51.391407,31/12/2014 09:47:50,113918054,12Elbestreet,@julieo25 Not yet got tics for Liverpool waiti...,Kingston upon Thames,NaN,0,2014-12-31 09:47:50
1,550226827944660992,NaN,-0.014891,51.354042,-0.014891,51.354042,31/12/2014 09:47:53,55872342,MissGee_Pee,If it doesn't make you happy by December 31st....,Croydon,NaN,0,2014-12-31 09:47:53
2,550226837537058816,NaN,0.006698,51.532020,0.006698,51.532021,31/12/2014 09:47:55,147266450,vinita_ramtri,@westfieldstrat thanks; and are you open tomor...,Newham,NaN,0,2014-12-31 09:47:55
3,550227214227505152,NaN,0.214417,51.582535,0.214417,51.582535,31/12/2014 09:49:25,518928608,CallumGordon7,@WestHam_Central but he is,Havering,NaN,0,2014-12-31 09:49:25
4,550227602011856896,NaN,-0.104869,51.509530,-0.104869,51.509529,31/12/2014 09:50:57,151094999,HevnKISZ,@JJJ_Jeeppy >> # XD,Southwark,,0,2014-12-31 09:50:57


In [4]:
tweets["datetime"] = pd.to_datetime(tweets.MESSAGEDATE)

In [5]:
tweets["datetime_to_nearest_hour"] = tweets.datetime.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour))

Remove non-plaintext elements from tweets

In [6]:
def parse_tweets(tweet):

    tweet = re.sub('@[^\s]+','',tweet).strip()
    tweet = re.sub('#[^\s]+','',tweet).strip()
    tweet = re.sub(r'https?:\/\/.*[\r\n]*','',tweet).strip()
    tweet = " ".join(re.findall("[a-zA-Z]+", tweet))
    
    return tweet

In [7]:
tweets["parsed_tweets"] = tweets.MESSAGETEXT.apply(lambda tweet: parse_tweets(tweet))

In [8]:
tweets.head()

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET,datetime,datetime_to_nearest_hour,parsed_tweets
0,550226818624942080,NaN,-0.262150,51.391407,-0.262150,51.391407,31/12/2014 09:47:50,113918054,12Elbestreet,@julieo25 Not yet got tics for Liverpool waiti...,Kingston upon Thames,NaN,0,2014-12-31 09:47:50,2014-12-31 09:00:00,Not yet got tics for Liverpool waiting till Fr...
1,550226827944660992,NaN,-0.014891,51.354042,-0.014891,51.354042,31/12/2014 09:47:53,55872342,MissGee_Pee,If it doesn't make you happy by December 31st....,Croydon,NaN,0,2014-12-31 09:47:53,2014-12-31 09:00:00,If it doesn t make you happy by December st le...
2,550226837537058816,NaN,0.006698,51.532020,0.006698,51.532021,31/12/2014 09:47:55,147266450,vinita_ramtri,@westfieldstrat thanks; and are you open tomor...,Newham,NaN,0,2014-12-31 09:47:55,2014-12-31 09:00:00,thanks and are you open tomorrow at all
3,550227214227505152,NaN,0.214417,51.582535,0.214417,51.582535,31/12/2014 09:49:25,518928608,CallumGordon7,@WestHam_Central but he is,Havering,NaN,0,2014-12-31 09:49:25,2014-12-31 09:00:00,but he is
4,550227602011856896,NaN,-0.104869,51.509530,-0.104869,51.509529,31/12/2014 09:50:57,151094999,HevnKISZ,@JJJ_Jeeppy >> # XD,Southwark,,0,2014-12-31 09:50:57,2014-12-31 09:00:00,XD


In [10]:
# tweets.groupby("datetime_to_nearest_hour")["parsed_tweets"].apply(lambda s: "%s" % ' '.join(s))
# tweets_by_hour = pd.DataFrame({"message_text_for_hour" : tweets.groupby("datetime_to_nearest_hour")["parsed_tweets"].apply(lambda s: "%s" % ' '.join(s))}).reset_index()

In [47]:
# need to get pos tag
class StemmerTokenizer(object):

    def __init__(self):
        self.stemmer = PorterStemmer()
        
    def __call__(self, doc):
        is_noun = lambda pos: pos[:2] == 'NN'
        doc = " ".join(re.findall("[a-zA-Z]+", doc))
        tokenized = nltk.word_tokenize(doc)
        nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 
        for b in [self.stemmer.stem(t) for t in word_tokenize(doc)]:
            if b == "thi":
                print(doc)
        return [self.stemmer.stem(t) for t in word_tokenize(doc)]

In [48]:
idf_by_hourly_interval = {}

for hour_interval in tweets.datetime_to_nearest_hour.unique():
    clf = TfidfVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1, 3), stop_words=set(stopwords.words('english')))
    clf.fit(tweets[tweets["datetime_to_nearest_hour"] == hour_interval]["parsed_tweets"])
    word2idf = dict(zip(clf.get_feature_names(), clf.idf_))
    sorted_idf = sorted(word2idf.items(), key=lambda x: x[1], reverse=True)

    idf_by_hourly_interval[hour_interval] = {
        "hour_interval": hour_interval,
        "sorted_idf": sorted_idf
    }
    
    print("Completed", hour_interval)

print("Finished!")

if any of my family downstairs are reading this i d love a cup of tea
omg omg omg there s this fine as man in the airport i m waiting for him to sin all over me so we can go to heaven
i m so utterly in love with harry i didn t think it was ever possible to have a heart full of this much love for only one person
commander ni rt thanks for two trophies this year many more to come this
commander ni rt thanks for two trophies this year many more to come this
i m actually looking forward to this weekend for the fact there s nothing going on festive fatigue has definitely hit
yes this weekend and the th you could try for discounted shoes
this is like my th room key
icy this morning have a great new years eve shepherd s bush london w
days make this one count
year old shoots mother in walmart this is madness less guns equals less people getting shot
if only london was this quiet all year round
kw output and this is what the gas boiler outputs to the outside
probably the funniest vine i ve seen

mornings like this mountsfield park
lovely stroll to work this morning in
this is my champagne highlight visit in henriot
Completed 2014-12-31T09:00:00.000000000
ijn will be x better than this year
first new years in england in years wonder how this will pan out
eat this as a treat x x
i m really not about this waking up and still being drunk life tbh
this little one says hello and she misses you
we re heading up the london eye this morning the eye also known as the millennium wheel opened in
this is the way to do it shows on nye i m healthy i m working and i m doing what i love twice happy new year friends
rt dk who this is but you gat this crush wcw
rt dk who this is but you gat this crush wcw
church this evening
this year i was the biggest witch every boy that crossed my path hates me
this year i m not gonna steal peoples girlfiends
saw this and thought of you could make a killing on this
saw this and thought of you could make a killing on this
ly love ya more you know this boo
oh n

my old man calls me this morning and after our normal talk he says he hopes i m not getting fat is it my fault
came home to this
who s this spurs player
have all the relevant vines ready to go once my game goes to shit this morning is there a banter specific one
can t believe my cousins wedding was this time last year
wore my pyjamas to work this morning
the view from the summit was amazing but pictures don t do justice this is where all our phones
lmao ah why rt my only regret this year was having sex on the th of october
this year has been shit can t wait to leave it behind me
i am not bothered to go down stairs get some food or a drink ugh this is what life has brung me too
left work this morning it s morning beckenham bromley united kingdom
going hard and listening to royal blood this morning happy new years eve people
safe to say i m much happier on the last days of this year let s just end it nicely and begin the new one
i could never get bored of waking up to this view
bitch don

do you know how funny this is
this woman is like a dream come true
found my soulmate in there s no way this year wasn t good to me
cheers darling not even i can repair this x
drown this year in a bottle of rum
what the fuck is this
i will slide in at least one girl s dms before this year ends
Completed 2014-12-31T04:00:00.000000000
i won t even lie i ll change your life in if you let me if you re a girl that s preeing me i can do this
i actually have zero planned for this evening
haha at this hour
who is this girl elena katherine trip sorry not anyones business just a stupid comment
today is the last day of this dreadful year and i m leaving this niggas there he is a part of my past
today is the last day of this dreadful year and i m leaving this niggas there he is a part of my past
it s this one guy i fell in love with but then he turnt out to be a fuck nigga so fuck that nigga
girls know wat they re doin when the do this shit
saw this in a pub its brilliant fox and firkin
this is gon

they said gmb truncated democracy am asking if they would be proud of this shagari govt
fuck a photo shoot we do this every night
every one gives it i m gonna change for the new yrs im gonna do this n that fuck all that u ant changing ull still do all the shit lol
this is definitely up there with the best vine of
this last episode of mic is nonsense
such drips is this msn ffs back to bebo
but in this case the aspirant was using it as a cardinal agenda in governance running lagos is more than that
one of my fave blogger statements made this year by
shooting lesson with the birthday boy i m finally getting the hang of this holland holland
i am not entering on bad terms with anyone my twitter is off private see this as an apology i forgive you too
this barber shop is too joke barbers are getting on that courvoisier like its
or even community work nothing wrong with this but it implies that people with disabilities are of little value we are more than that
i ll turn this year no thank you


...

i was thinking this but still scouting for that foreignnnnnn rod
do you have any shows in london this year pleeeease
i hope this year is better than the last
wish you happy new year i pray this year bring lot of happiness lot of success in your life
the forces combining against me we shall see who s gona win this battle
come to england so you can do this with me gah
so far this evening booked a train coach and hotel that i m not entirely sure i m going to use
can t wait to see this fucker on tuesday
this year i ll start rd year at artsed and perform my first public show in the autumn that s scary to say omg
watch this brilliant fan made video of new song only one
hey this isn t an exam you know kmt
what i ve learnt this year
could i have i don t feel attached to this name
lmaoo nd of january already uno this year is basically finished
i swear it s to do with this thing i have for the tfl and general travel
i just talk a load of crap at this time of evening
i also turn this year
nah thi

ghana and jamaica again this year my saving needs to be on fleek
if this is in nigeria some ignorant people will probably form a barricade to prevent his trial just like ibori
about to start this cardio pure gym greenwich
this is the only picture in existence of me genuinely laughing
getting around london is very easy this shot was taken from the heathrow express
this video does last night no justice at all happy new year everyone
and j mans receivers are hardly making plays this could heat up in nd half least i hope it does
this baby s crying is giving me a migraine
too many arguments about this and now i have a headache
jas we all know who this really is
this is not ok abby is not a happy bunny
this is what happens when i m drunk
this movie is phenomenal it honestly deserves all of the awards that it has been given
if you re buff in everyone s eyes and you re perfect then you can call others ugly if you don t fit this criteria than keep it shut lol
whoever is doing this where people 

delete this you fucking cunt i hope your mum gets acid splashed on her face
on god i know im not the only who does this lmfaooo
why would you do this to me
swear down watch this save save of the season from greeno
wait wtf she was dead now she alive what kinda magic is this
did szczesny take any selfies at the end of this game
one fucking game joint top clean sheets last season excellent saves in other games this season two mistakes and his awful
liverpool are so shit how did they not lose this game is beyond me
let s make sure we save the nhs this may
my beautiful girl britney spears i am so proud of her i know one day this picture will turn into
the elevation on this makes me wet
still laughing comment in this video primrose
who did this
loool man drank his water after he conceded is this tennis heyehegyevgeehef
Completed 2015-01-01T23:00:00.000000000
what on earth is this
lord voldemort via who wants to buy me this
i swear i m going to take the bull by the horns this year
hope you g

god this film is spectacular in every way the story is faultless and heath ledgers joker is phenomenal
i m actually scared for this season
lol so there s this white boy who has been stressin my soul since i was with shit like come mine be mine im different
watched it this morning you re not wrong
harry kane ain t messing this season
loooool it s a new year n thursday so this is my se south bermondsey
bbw or slim girls what does bbw mean been meaning to ask this
this is true
certainly would loads of pressure now mind the media especially will wring this for every last drop of sentimental fairytale
this is genuinely what i looked like this morning the resemblance is uncanny
this is genuinely what i looked like this morning the resemblance is uncanny
best weekend for football so far this season
i have this same issue rt too many lagos people follow me why can t abuja people be following me
excited for this summer
kleberson won a world cup and messi hasn t yet you people run this shit bant

when asked what my new years resolution was going to be for i showed them this picture for
this boy gets away with murder by looking like this
this boy gets away with murder by looking like this
his defo selling out o this year
st one episode wasn t expecting it to be this good take it you are a fan
this is when i thought chelsea were gonna be on flames
Completed 2015-01-01T22:00:00.000000000
this year
this nicca always on my tl gweh
how am i dancing to salty tic toc at this time like say i aint got better things to do
the way everton have fell off this season effect
omnomnomnom this is before going through them lobsters still smiling post lobster annihilation
i ve been burnt in different places by this hot water bottle ffs
for why do i go to sleep at this time
could never live alone absolutely hate being in this flat alone
because of this extra income that leads to more spending the multiplier effect suttem like that
up with this one she s scared of the dark so i have to stay awake wi

In [12]:
for interval in idf_by_hourly_interval.keys():
    print(interval)
    print(idf_by_hourly_interval[interval]["sorted_idf"][-5:])

2014-12-31T09:00:00.000000000
[('happi', 3.862553552419634), ('new year', 3.703125815491532), ('new', 3.503412515985708), ('thi', 3.4778792139805432), ('year', 3.181828891858246)]
2014-12-31T10:00:00.000000000
[('happi', 4.029398290247109), ('new year', 3.8507065015037334), ('thi', 3.7149049603446715), ('new', 3.64591208885772), ('year', 3.2184680740307807)]
2014-12-31T08:00:00.000000000
[('day', 3.8409524650491713), ('new year', 3.7508013680548737), ('thi', 3.7508013680548737), ('new', 3.5207849374529023), ('year', 3.300384872084443)]
2014-12-31T06:00:00.000000000
[('today', 3.878395566849449), ('new year', 3.627081138568543), ('new', 3.5217206229107165), ('thi', 3.472930458741285), ('year', 3.472930458741285)]
2014-12-31T07:00:00.000000000
[('work', 3.894478539848818), ('thi', 3.8581108956779433), ('new', 3.7245795030534206), ('london', 3.693807844386667), ('year', 3.5793974932089228)]
2014-12-31T04:00:00.000000000
[('like', 4.4094961844768505), ('sleep', 4.237645927550192), ('thi', 

In [13]:
idf_by_hourly_interval[np.datetime64('2015-01-01T21:00:00.000000000')]["sorted_idf"][-10:]

[('go', 4.488095834480246),
 ('kill', 4.488095834480246),
 ('die', 4.440467785490991),
 ('new', 4.289644895756409),
 ('like', 4.250424182603126),
 ('ronni', 4.032422030789336),
 ('wa', 3.934304174830812),
 ('year', 3.915785127063575),
 ('eastend', 3.7828272933879563),
 ('thi', 3.731935686091567)]

In [ ]:
('eastend', 3.7828272933879563),
('ronni', 4.032422030789336),
('kill', 4.488095834480246),
('die', 4.440467785490991),
('emma', 4.735931998384828),
('luci', 4.767680696699408),
('phil', 5.023614070836609)
('nick', 5.539080074159101)
('denis', 5.110625447826238)
('dead', 5.292947004620193)

In [15]:
idf_by_hourly_interval[np.datetime64('2015-01-01T18:00:00.000000000')]["sorted_idf"][-10:]

[('wa', 4.331218802850445),
 ('spur', 4.324345923562682),
 ('harri', 4.317519958492283),
 ('happi', 4.284072024424743),
 ('new year', 4.109455514005193),
 ('new', 3.92345748248193),
 ('chelsea', 3.9007807738109004),
 ('kane', 3.8655348347492255),
 ('year', 3.7904996488063114),
 ('thi', 3.5179923743737658)]

In [37]:
idf_by_hourly_interval[np.datetime64('2015-01-01T17:00:00.000000000')]["sorted_idf"][-10:]

[('happi new', 4.403730402737885),
 ('get', 4.281128080645553),
 ('go', 4.281128080645553),
 ('london', 4.146395486675397),
 ('happi', 4.042717057200555),
 ('wa', 4.012864094050874),
 ('thi', 3.8950810583944904),
 ('new year', 3.8317687789801633),
 ('new', 3.7105832221779402),
 ('year', 3.453248306115451)]

In [41]:
idf_by_hourly_interval[np.datetime64('2015-01-01T18:00:00.000000000')]["sorted_idf"][-10:]

[('wa', 4.331218802850445),
 ('spur', 4.324345923562682),
 ('harri', 4.317519958492283),
 ('happi', 4.284072024424743),
 ('new year', 4.109455514005193),
 ('new', 3.92345748248193),
 ('chelsea', 3.9007807738109004),
 ('kane', 3.8655348347492255),
 ('year', 3.7904996488063114),
 ('thi', 3.5179923743737658)]

In [42]:
idf_by_hourly_interval[np.datetime64('2015-01-01T19:00:00.000000000')]["sorted_idf"][-100:]

[('wait', 5.802280832982721),
 ('ye', 5.802280832982721),
 ('could', 5.77242786983304),
 ('absolut', 5.743440332959787),
 ('believ', 5.743440332959787),
 ('chadli', 5.743440332959787),
 ('ever', 5.743440332959787),
 ('lane', 5.715269455993091),
 ('last night', 5.715269455993091),
 ('better', 5.687870481804977),
 ('home', 5.687870481804977),
 ('point', 5.687870481804977),
 ('pleas', 5.609908940335265),
 ('rt', 5.609908940335265),
 ('peopl', 5.585216327744893),
 ('player', 5.585216327744893),
 ('let', 5.5611187761658325),
 ('tonight', 5.5611187761658325),
 ('beat', 5.51459876053094),
 ('cahil', 5.51459876053094),
 ('score', 5.51459876053094),
 ('take', 5.448640792739143),
 ('much', 5.427587383541311),
 ('shit', 5.427587383541311),
 ('way', 5.427587383541311),
 ('hope', 5.386765389021055),
 ('na', 5.386765389021055),
 ('realli', 5.386765389021055),
 ('right', 5.386765389021055),
 ('best', 5.366962761724875),
 ('feel', 5.366962761724875),
 ('hi', 5.366962761724875),
 ('first', 5.3475446758

In [43]:
['arsen','goal','lampard','wenger','liverpool','team','chelsea','game','kane','spur','harri','tottenham','sunderland','player','leagu','footbal','arsen','mourinho','play','beat','cahil','score']

['arsen',
 'goal',
 'lampard',
 'wenger',
 'liverpool',
 'team',
 'chelsea',
 'game',
 'kane',
 'spur',
 'harri',
 'tottenham',
 'sunderland',
 'player',
 'leagu',
 'footbal',
 'arsen',
 'mourinho',
 'play',
 'beat',
 'cahil',
 'score']

In [45]:
idf_by_hourly_interval[np.datetime64('2014-12-31T19:00:00.000000000')]["sorted_idf"][-100:]

[('next', 5.541164856012179),
 ('still', 5.541164856012179),
 ('thing', 5.541164856012179),
 ('want', 5.541164856012179),
 ('de', 5.501944142858898),
 ('let', 5.501944142858898),
 ('need', 5.501944142858898),
 ('readi', 5.501944142858898),
 ('think', 5.501944142858898),
 ('ever', 5.464203814876051),
 ('us', 5.464203814876051),
 ('friend', 5.427836170705175),
 ('gon', 5.427836170705175),
 ('gon na', 5.427836170705175),
 ('good luck run', 5.427836170705175),
 ('hi', 5.427836170705175),
 ('luck run', 5.427836170705175),
 ('veri', 5.427836170705175),
 ('fuck', 5.392744850893906),
 ('last', 5.392744850893906),
 ('start', 5.392744850893906),
 ('day', 5.358843299218225),
 ('greater', 5.358843299218225),
 ('greater london', 5.358843299218225),
 ('way', 5.358843299218225),
 ('work', 5.358843299218225),
 ('would', 5.358843299218225),
 ('follow', 5.326053476395233),
 ('forev matter', 5.326053476395233),
 ('forev matter wat', 5.326053476395233),
 ('good luck', 5.326053476395233),
 ('keep thi', 5.3

In [51]:
interval_by_idf = {}

for interval in idf_by_hourly_interval.keys():
    top_idf = idf_by_hourly_interval[interval]["sorted_idf"][-1]
    
    if top_idf in interval_by_idf:
        interval_by_idf[top_idf].append(interval)
    else:
        interval_by_idf[top_idf] = [interval]

In [52]:
interval_by_idf

{('new',
  2.6085698568240367): [numpy.datetime64('2014-12-31T20:00:00.000000000')],
 ('new',
  3.0962099766662408): [numpy.datetime64('2015-01-01T15:00:00.000000000')],
 ('thi',
  3.5179923743737658): [numpy.datetime64('2015-01-01T18:00:00.000000000')],
 ('thi',
  3.7127340504146367): [numpy.datetime64('2015-02-01T02:00:00.000000000')],
 ('thi',
  3.729622686356338): [numpy.datetime64('2015-02-01T01:00:00.000000000')],
 ('thi',
  3.731935686091567): [numpy.datetime64('2015-01-01T21:00:00.000000000')],
 ('thi',
  3.8017642113737127): [numpy.datetime64('2014-12-31T05:00:00.000000000')],
 ('thi',
  3.8129560679683903): [numpy.datetime64('2015-02-01T03:00:00.000000000')],
 ('thi',
  3.8319609957397502): [numpy.datetime64('2015-02-01T00:00:00.000000000')],
 ('thi',
  3.8370423212933136): [numpy.datetime64('2015-01-01T23:00:00.000000000')],
 ('year',
  1.7824502593219131): [numpy.datetime64('2015-01-01T00:00:00.000000000')],
 ('year',
  2.1929179480464294): [numpy.datetime64('2015-01-01T01:

In [46]:
tweets[tweets.datetime_to_nearest_hour == np.datetime64('2015-02-01T02:00:00.000000000')].parsed_tweets

94778    Squirtle normally but ruby sapphire is over po...
94779                      Sometimes it s not good to pree
94780    I swear sleep is so important Sleep over food ...
94781                                                     
94782    laying in bed in the dark and I decide to star...
94783                                                     
94784                           That s if its two triangle
94785                         lee evans is toooooooo funny
94786                                    I want some pizza
94787                Loool i was like you then she woke up
94788    how do i show i am sad i keep quiet and head d...
94789                      bieber stop being so attractive
94790                                           manda fota
94791                   Hepiniz amaa da cook bilmisiniz ha
94792                                       massive scenes
94793    Conseguiu ser gato a vida inteira e ainda cont...
94794                             Lidge anyone can have 